In [11]:
import sqlite3
import os
import sys

In [18]:
# Creates the schedule table
con = sqlite3.connect("example.db")
cur = con.cursor()
cur.execute('''CREATE TABLE schedule (gameID text, date DATE, season text, hometeam text, awayteam text, winner int)''')

game_log_dir = './data/game_logs/'
logs = os.listdir(game_log_dir)
for file in logs: 
    if file[0] == '.': 
        continue
    season = file[2:6]
    with open(os.path.join(game_log_dir, file), 'r') as f: 
        for line in f: 
            line = line.split(',')
            date = line[0].strip('\"')
            date = date[:4] + '-' + date[4:6] + '-' + date[6:]
            game_id = line[6].strip('\"') + line[0].strip('\"') + line[1].strip('\"')
            hometeam = line[6].strip('\"')
            awayteam = line[3].strip('\"')
            home_score = int(line[10])
            away_score = int(line[9])
            winner = home_score > away_score
            cur.execute(f"INSERT INTO schedule VALUES ('{game_id}','{date}','{season}', '{hometeam}','{awayteam}',{winner})")

con.commit()


In [14]:
from pprint import pprint

def insert_lineup(con, data):
    columns = [
        'p1ID', 'p1PC', 'p2ID', 'p2PC',
        'p3ID', 'p3PC', 'p4ID', 'p4PC',
        'p5ID', 'p5PC', 'p6ID', 'p6PC',
        'p7ID', 'p7PC', 'p8ID', 'p8PC',
        'p9ID', 'p9PC', 'p10ID', 'p10PC',
        'p11ID', 'p11PC', 'p12ID', 'p12PC',
        'p13ID', 'p13PC', 
    ]
    for col in columns: 
        if col not in data: 
            if 'PC' in col: 
                data[col] = 0
            else: 
                data[col] = 'NULL'
    con.execute(
    f'''INSERT INTO lineup VALUES ('{data['gameID']}', '{data['date']}', '{data['team']}',
                '{data['bo1ID']}', '{data['bo2ID']}',
                '{data['bo3ID']}', '{data['bo4ID']}',
                '{data['bo5ID']}', '{data['bo6ID']}',
                '{data['bo7ID']}', '{data['bo8ID']}',
                '{data['bo9ID']}',
                '{data['p1ID']}', {data['p1PC']}, 
                '{data['p2ID']}', {data['p2PC']},
                '{data['p3ID']}', {data['p3PC']}, 
                '{data['p4ID']}', {data['p4PC']}, 
                '{data['p5ID']}', {data['p5PC']}, 
                '{data['p6ID']}', {data['p6PC']},
                '{data['p7ID']}', {data['p7PC']}, 
                '{data['p8ID']}', {data['p8PC']}, 
                '{data['p9ID']}', {data['p9PC']}, 
                '{data['p10ID']}', {data['p10PC']},
                '{data['p11ID']}', {data['p11PC']}, 
                '{data['p12ID']}', {data['p12PC']}, 
                '{data['p13ID']}', {data['p13PC']}
    );''')
    
def parse_evx(path, con): 
    first = 1
    with open(path, 'r') as f: 
        for idx, line in enumerate(f):
            line = line.strip('\n')
            line = line.split(',')
            if len(line) == 0: 
                    break
            match line[0]:
                case 'id': 
                    if not first: 
                        insert_lineup(con, home_data)
                        insert_lineup(con, away_data)
                    first = 0
                    home_data = {}
                    away_data = {}
                    h_num_pitchers = 1
                    h_pc = 0
                    a_num_pitchers = 1
                    a_pc = 0
                    home_data['gameID'] = line[1]
                    away_data['gameID'] = line[1]
                case 'info': 
                    if line[1] == 'visteam': 
                        away_data['team'] = line[2]
                    elif line[1] == 'hometeam': 
                        home_data['team'] = line[2]
                    elif line[1] == 'date': 
                        home_data['date'] = line[2][:4] + '-' + line[2][5:7] + '-' + line[2][8:]
                        away_data['date'] = line[2][:4] + '-' + line[2][5:7] + '-' + line[2][8:]
                case 'start': 
                    if line[3] == '0': 
                        data = away_data
                    else: 
                        data = home_data
                    bo = int(line[4])

                    id = line[1]
                    fp = line[-1]
                    if fp == '1': 
                        data['p1ID'] = id
                        data['p1PC'] = 0
                    if bo != '0': 
                        data[f'bo{bo}ID'] = id
                case 'sub':
                    fp = line[-1]
                    id = line[1]
                    if fp == '1':
                        if line[3] == '0': 
                            data = away_data
                            a_num_pitchers += 1
                            r = a_num_pitchers
                        else: 
                            data = home_data
                            h_num_pitchers += 1
                            r = h_num_pitchers
                        data[f'p{r}ID'] = id
                        data[f'p{r}PC'] = 0
                    
                case 'play': 
                    if line[2] == '0': 
                        data = home_data
                        r = h_num_pitchers
                    else: 
                        data = away_data
                        r = a_num_pitchers
                    pc = int(line[4][0]) + int(line[4][1])
                    data[f'p{r}PC'] += pc



In [1]:
import os

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute(
    '''CREATE TABLE lineup (gameID text, 
                date DATE, team text, 
                bo1ID text, bo2ID text,
                bo3ID text, bo4ID text,
                bo5ID text, bo6ID text,
                bo7ID text, bo8ID text,
                bo9ID text,
                p1Id text, p1PC int, 
                p2ID text, p2PC int,  
                p3ID text, p3PC int, 
                p4ID text, p4PC int,
                p5ID text, p5PC int,
                p6ID text, p6PC int,
                p7ID text, p7PC int,
                p8ID text, p8PC int,
                p9ID text, p9PC int,
                p10ID text, p10PC int,
                p11ID text, p11PC int, 
                p12ID text, p12PC int, 
                p13ID text, p13PC int )
    ''')

play_by_play_dir = './data/playbyplay/'
for direc in os.listdir(play_by_play_dir): 
    for evx in os.listdir(play_by_play_dir + direc): 
        parse_evx(play_by_play_dir + direc + '/' + evx, con=cur)





NameError: name 'sqlite3' is not defined

In [10]:
import pandas as pd
lahman_batting_path = './data/lahman/core/Batting.csv'
df = pd.read_csv(lahman_batting_path)
df = df.drop(columns=['lgID'])
people_path = './data/lahman/core/People.csv'

def map_ids(people_path):
    people_map = {}
    with open(people_path) as f: 
        for line in f: 
            line = line[:-1]
            line = line.split(',')
            if line[0] == 'playerID':
                continue
            people_map[line[0]] = line[-2]
    return people_map

lahman_to_retrosheet = map_ids(people_path)
df['playerID'] = df['playerID'].apply(lambda x: lahman_to_retrosheet[x])


sub_df = df.groupby(['playerID', 'yearID']).sum()
sub_df = sub_df.drop(columns=['stint'])
con = sqlite3.connect("baseball.db")
cur = con.cursor()

sub_df.to_sql('batters', con=con, index=False)

con.close()



In [73]:
import pandas as pd

con = sqlite3.connect("example.db")
cur = con.cursor()
lahman_pitching_path = './data/lahman/core/Pitching.csv'
df = pd.read_csv(lahman_pitching_path)
df = df.drop(columns=['lgID'])
people_path = './data/lahman/core/People.csv'

def map_ids(people_path):
    people_map = {}
    with open(people_path) as f: 
        for line in f: 
            line = line[:-1]
            line = line.split(',')
            if line[0] == 'playerID':
                continue
            people_map[line[0]] = line[-2]
    return people_map

lahman_to_retrosheet = map_ids(people_path)
df['playerID'] = df['playerID'].apply(lambda x: lahman_to_retrosheet[x])

sub_df = df.drop(columns=['stint', 'teamID'])
sub_df = sub_df.groupby(['playerID', 'yearID']).sum().reset_index()

sub_df.to_sql('pitchers', con=con, index=False)
# cur.execute("""SELECT playerID,yearID, SUM(W) from 'pitchers' GROUP BY playerID, yearID""")
con.commit()
print(sub_df)
con.close()

       playerID  yearID  W  L   G  GS  CG  SHO  SV  IPouts  ...  IBB  WP  HBP  \
0      aardd001    2004  1  0  11   0   0    0   0      32  ...  0.0   0  2.0   
1      aardd001    2006  3  0  45   0   0    0   0     159  ...  0.0   1  1.0   
2      aardd001    2007  2  1  25   0   0    0   0      97  ...  3.0   2  1.0   
3      aardd001    2008  4  2  47   0   0    0   0     146  ...  2.0   3  5.0   
4      aardd001    2009  3  6  73   0   0    0  38     214  ...  3.0   2  0.0   
...         ...     ... .. ..  ..  ..  ..  ...  ..     ...  ...  ...  ..  ...   
45627  zuveg101    1958  2  2  45   0   0    0   7     207  ...  3.0   2  6.0   
45628  zuveg101    1959  0  1   6   0   0    0   0      39  ...  0.0   1  0.0   
45629  zycht001    2015  0  0  13   1   0    0   0      55  ...  0.0   1  2.0   
45630  zycht001    2016  1  0  12   0   0    0   0      41  ...  2.0   0  1.0   
45631  zycht001    2017  6  3  45   0   0    0   1     122  ...  3.0   1  5.0   

       BK    BFP  GF   R   

In [17]:
plays_path = './retrosheet_parsing/data/plays.csv'
plays_data = pd.read_csv(plays_path)
def add_outcome(x): 
    if "G" in x: 
        return "groundout"
    elif "F" in x: 
        return "flyout"
    elif "K" in x: 
        return "strikeout"
    elif "S" in x: 
        return "single"
    elif "D" in x: 
        return "double"
    elif "T" in x: 
        return "triple"
    elif "W" in x or "HP" in x: 
        return "walk"
    elif "H" in x: 
        return "homerun"
    else: 
        return "other"
plays_data["outcome"] = plays_data["play_str"].apply(add_outcome)
data = plays_data[["pitcher", "player_id", "outcome"]]

con = sqlite3.connect("example.db")
cur = con.cursor()

data.to_sql('plays', con=con, index=False)

con.close()

In [72]:
states = [
        [0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],
        [0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1],
        [0,0,0,2],[1,0,0,2],[0,1,0,2],[0,0,1,2],[1,1,0,2],[1,0,1,2],[0,1,1,2],[1,1,1,2],
        [0,0,0,3]
    ]
transitions = pd.DataFrame([], columns=['pre_1', 'pre_2', 'pre_3', 'pre_outs', 'post_1','post_2', 'post_3', 'post_outs', 'runs', 'outcome'])
transitions['pre_1'] = (plays_data['pre_state'].apply(lambda x: states[x-1][0]))
transitions['pre_2'] = (plays_data['pre_state'].apply(lambda x: states[x-1][1]))
transitions['pre_3'] = (plays_data['pre_state'].apply(lambda x: states[x-1][2]))
transitions['pre_outs'] = (plays_data['pre_state'].apply(lambda x: states[x-1][3]))
transitions['post_1'] = (plays_data['post_state'].apply(lambda x: states[min(x-1, 24)][0]))
transitions['post_2'] = (plays_data['post_state'].apply(lambda x: states[min(x-1, 24)][1]))
transitions['post_3'] = (plays_data['post_state'].apply(lambda x: states[min(x-1, 24)][2]))
transitions['post_outs'] = (plays_data['post_state'].apply(lambda x: states[min(x-1, 24)][3]))
transitions['runs'] = plays_data['play_runs']
transitions['outcome'] = plays_data['outcome']
con = sqlite3.connect("example.db")
cur = con.cursor()

transitions.to_sql('transitions', con=con, index=False)

con.close()
# data = plays_data['pre_state'].apply(lambda x: pd.Series(states[1], index=['pre_1', 'pre_2' 'pre_3', 'pre_outs']), axis=1)
# data

   A  B
0  4  9
1  4  9
2  4  9
   foo  bar
0  NaN  NaN
1  NaN  NaN
2  NaN  NaN


# Create the helper functions

In [42]:
con = sqlite3.connect("baseball.db")
cur = con.cursor()

def get_prev_season(cur, date): 
    cur.execute(f"SELECT season FROM schedule WHERE date > '{date}' ORDER BY season ASC LIMIT 1")
    rows = cur.fetchall()
    season_after = int(rows[0][0])

    cur.execute(f"SELECT season FROM schedule WHERE date < '{date}' ORDER BY season DESC LIMIT 1")
    rows = cur.fetchall()
    season_before = int(rows[0][0])
    if season_after == season_before: 
        return season_after - 1
    return season_before


print(get_prev_season(cur, '2014-10-30'))

2014


In [48]:
import pandas as pd
con = sqlite3.connect("baseball.db")
cur = con.cursor()

def get_last_pitchers(team_id, date, n): 
    """
    return the last n starting pitchers and last n relievers used in previous games and the dates of their games.
    """
    q = f"""SELECT date,p1ID,p1PC,p2ID,p2PC,p3ID,p3PC,p4ID,p4PC,p5ID,p5PC,p6ID,p6PC,p7ID,p7PC,p8ID,
p8PC,p9ID,p9PC,p10ID,p10PC,p11ID,p11PC,p12ID,p12PC,p13ID,p13PC FROM lineup WHERE date < 
'{date}'  and team == '{team_id}' ORDER BY date DESC LIMIT {n}"""
    return pd.read_sql_query(q, con)

get_last_pitchers('PHI', '2013-07-25', 5)

,date,p1Id,p1PC,p2ID,p2PC,p3ID,p3PC,p4ID,p4PC,p5ID,...,p9ID,p9PC,p10ID,p10PC,p11ID,p11PC,p12ID,p12PC,p13ID,p13PC
0,2013-07-24,lannj001,63,ramij002,28,defrj001,26,diekj001,18,NULL,...,NULL,0,NULL,0,NULL,0,NULL,0,NULL,0
1,2013-07-23,pettj001,58,valdr002,12,defrj001,5,diekj001,2,garcl005,...,NULL,0,NULL,0,NULL,0,NULL,0,NULL,0
2,2013-07-21,lee-c003,66,basta001,11,papej001,8,NULL,0,NULL,...,NULL,0,NULL,0,NULL,0,NULL,0,NULL,0
3,2013-07-20,hamec001,69,ramij002,17,diekj001,3,garcl005,16,basta001,...,NULL,0,NULL,0,NULL,0,NULL,0,NULL,0
4,2013-07-19,kendk001,70,garcl005,13,diekj001,10,defrj001,9,NULL,...,NULL,0,NULL,0,NULL,0,NULL,0,NULL,0


In [80]:
con = sqlite3.connect("baseball.db")

def get_batter_stats(con,batter_id, date): 
    cur = con.cursor()
    season = get_prev_season(cur, date)
    q = f"""SELECT * FROM batters WHERE season <=
'{season}' and playerID == '{batter_id}' ORDER BY season DESC LIMIT 1"""
    return pd.read_sql_query(q, con)

get_batter_stats(con, 'pench001', '2014-05-08')

,playerID,season,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,pench001,2013,162,629,91,178,35,5,27,99.0,22.0,3.0,52,115.0,3.0,3.0,0.0,3.0,17.0
